In [ ]:
#!rm -rf /content/ML_course_IUPWARE2025

# Curso: Machine Learning para Pronóstico Hidrológico

<div style="text-align:center;">
    <img src="https://github.com/paulmunozpauta//Curso_ML_pronostico_hidrologico/blob/main/notebooks/static/imgs/Logo_course.png?raw=true" width="300">
    <p style="margin-top:10px;">
        Contacto: paul.andres.munoz@gmail.com
    </p>
    <p><a href="https://paulmunozpauta.github.io/paulmunozpauta/index.html" target="_blank">Website personal</a></p>
</div>




## ✅ Antes de comenzar: sigue estos 4 pasos para ejecutar el notebook en Google Colab


Paso 1. Clona el repositorio de GitHub con los notebooks y datos del curso.


In [ ]:
!git clone -- https://github.com/paulmunozpauta/Curso_ML_pronostico_hidrologico.git

Paso 2. Accede a la carpeta clonada.



In [ ]:
ls


In [ ]:
%cd Curso_ML_pronostico_hidrologico

In [ ]:
ls

Paso 3. Configura el entorno para ejecutar el código del curso.


In [ ]:
# Resolver un conflicto con Colab, nueva versión de numpy
!pip uninstall -y numpy
# Instalar las versión de numpy para el curso
!pip install numpy==1.24.4

In [ ]:
# Install Poetry
!pip install poetry
# Disable virtual environment creation (needed for Colab)
!poetry config virtualenvs.create false

🔁 Si la sesión se reinicia, repite los pasos 2 y 3.
➡️ Si no, continúa con el paso 4.



Paso 4. Instala los paquetes necesarios para el curso.


In [ ]:
%cd Curso_ML_pronostico_hidrologico
!poetry lock

In [ ]:
!poetry install --no-root

🧪 Ahora sí, empezamos con la parte práctica del curso


# Parte 2: Desarrollo de modelos hidrológicos con Machine Learning
En esta sesión, vamos a:

Desarrollar modelos de pronóstico para el caso de la cuenca de montaña ⛰️




## Importar bibliotecas









In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as dates
import os
import datetime
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from copy import deepcopy
import seaborn as sns
from sklearn.model_selection import GridSearchCV
import itertools
import random

def lagged_dataset(arr, num_steps, additional_arr, new_num_steps):
    num_columns = arr.shape[1]
    modified_rows = []
    excluded_data = []
    for i in range(num_steps, arr.shape[0]):
        prev_rows = arr[i - num_steps:i]
        current_row = arr[i]
        new_row = np.concatenate((prev_rows.flatten(), current_row))
        modified_rows.append(new_row)
    result_array = np.array(modified_rows)
    # Slicing the result_array to match the number of rows in modified_additional_arr
    if result_array.shape[0] > additional_arr.shape[0]:
        result_array = result_array[result_array.shape[0] - additional_arr.shape[0]:]

    modified_rows = []
    for i in range(new_num_steps, additional_arr.shape[0]):
        prev_rows = additional_arr[i - new_num_steps:i]
        current_row = additional_arr[i]
        excluded_data.append(current_row[-1])  # Store excluded data
        new_row = np.concatenate((prev_rows.flatten(), current_row[:-1]))  # Exclude last column
        modified_rows.append(new_row)

    modified_additional_arr = np.array(modified_rows)

    # Adjust dimensions by removing rows from result_array or modified_additional_arr
    min_rows = min(result_array.shape[0], modified_additional_arr.shape[0])
    result_array = result_array[-min_rows:]
    modified_additional_arr = modified_additional_arr[-min_rows:]
    excluded_data = np.array(excluded_data)[-min_rows:]

    # Concatenate result_array and modified_additional_arr
    final_result = np.concatenate((result_array, modified_additional_arr), axis=1)

    return final_result, np.array(excluded_data)[:, None]

def lagged_dataset_pron(arr, num_steps, additional_arr, new_num_steps, lead_time):
    num_columns = arr.shape[1]
    modified_rows = []
    excluded_data = []

    for i in range(num_steps, arr.shape[0]):
        prev_rows = arr[i - num_steps:i]
        current_row = arr[i]
        new_row = np.concatenate((prev_rows.flatten(), current_row))
        modified_rows.append(new_row)

    result_array = np.array(modified_rows)

    # Slicing the result_array to match the number of rows in modified_additional_arr
    if result_array.shape[0] > additional_arr.shape[0]:
        result_array = result_array[result_array.shape[0] - additional_arr.shape[0]:]

    modified_rows = []
    for i in range(new_num_steps, additional_arr.shape[0]):
        prev_rows = additional_arr[i - new_num_steps:i]
        current_row = additional_arr[i]
        excluded_data.append(current_row[-1])  # Store excluded data
        new_row = np.concatenate((prev_rows.flatten(), current_row))  # Include last column
        modified_rows.append(new_row)

    modified_additional_arr = np.array(modified_rows)

    # Adjust dimensions by removing rows from result_array or modified_additional_arr
    min_rows = min(result_array.shape[0], modified_additional_arr.shape[0])
    result_array = result_array[-min_rows:]
    modified_additional_arr = modified_additional_arr[-min_rows:]
    excluded_data = np.array(excluded_data)[-min_rows:]

    # Shift excluded_data by lead_time
    excluded_data = excluded_data[lead_time:]

    # Concatenate result_array and modified_additional_arr
    final_result = np.concatenate((result_array, modified_additional_arr), axis=1)

    # Resize final_result and excluded_data to have the same number of rows
    min_rows = min(final_result.shape[0], excluded_data.shape[0])
    final_result = final_result[:min_rows]
    excluded_data = excluded_data[:min_rows]

    return final_result, np.array(excluded_data)[:, None]


def calculate_hydro_metrics(simulations, evaluation):
    sim_mean = np.mean(simulations, axis=0, dtype=np.float64)
    obs_mean = np.mean(evaluation, dtype=np.float64)

    r_num = np.sum((simulations - sim_mean) * (evaluation - obs_mean),
                   axis=0, dtype=np.float64)
    r_den = np.sqrt(np.sum((simulations - sim_mean) ** 2,
                           axis=0, dtype=np.float64)
                    * np.sum((evaluation - obs_mean) ** 2,
                             dtype=np.float64))
    r = r_num / r_den
    # calculate error in spread of flow alpha
    alpha = np.std(simulations, axis=0) / np.std(evaluation, dtype=np.float64)
    # calculate error in volume beta (bias of mean discharge)
    beta = (np.sum(simulations, axis=0, dtype=np.float64)
            / np.sum(evaluation, dtype=np.float64))
    # calculate the Kling-Gupta Efficiency KGE
    kge = 1 - np.sqrt((r - 1) ** 2 + (alpha - 1) ** 2 + (beta - 1) ** 2)
    rmse = np.sqrt(np.mean((evaluation - simulations) ** 2,
                            axis=0, dtype=np.float64))
    pbias = (100 * np.sum(evaluation - simulations, axis=0, dtype=np.float64)
              / np.sum(evaluation))
    r2 = 1 - (np.sum((evaluation - simulations)**2) / np.sum((evaluation - np.mean(evaluation))**2))
    return kge, rmse, pbias, r2
np.random.seed(22)
random.seed(22)

## Seleccionar carpeta del proyecto


In [ ]:
folder = os.getcwd()+'/notebooks/data/'
folder

## Importar datos de precipitación 🌧️


### Precipitación satelital 🛰️



Leer datos de la cuenca de montaña ⛰️

In [ ]:

# Import satellite precipitation data
precipitation_satellite = pd.read_csv(folder + 'PERSIANN-CCS_UTC_daily_catchment_1.csv', sep=',')
# Rename columns
precipitation_satellite.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
# Convert 'Date' column to datetime format (without unnecessary dayfirst=True)
precipitation_satellite['Date'] = pd.to_datetime(precipitation_satellite['Date'], format='%Y-%m-%d')
# Set 'Date' as the index
precipitation_satellite.set_index('Date', inplace=True)
# Print first rows to verify
print(precipitation_satellite.head())

In [ ]:
precipitation_satellite

Calcular la precipitación anual 📆🌧️


In [ ]:
# Resample annual precipitation data
data_annual = precipitation_satellite.resample('Y', label='right', closed='right').sum()

Graficar la precipitación anual promedio 📊


In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
# Assuming dataset is a pandas DataFrame with labeled columns
data_annual.plot(kind='bar', ax=ax)
# Adding labels for the legend
ax.legend(title='Legend Title')
# Adding a label to the y-axis
plt.ylabel('Precipitation_satellite (mm)')
# Adjusting the position of the legend
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.6), ncol=6)
plt.show()

Calcular la precipitación anual promedio para todos los píxeles en la cuenca 🧩⛰️


In [ ]:
data_annual_average =  data_annual.mean(axis=1)
data_annual_average

Graficar la precipitación promedio (todos los píxeles) 📈


In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
# Assuming dataset is a pandas DataFrame with labeled columns
data_annual_average.plot(kind='bar', ax=ax)
# Adding labels for the legend
ax.legend(title='Legend Title')
# Adding a label to the y-axis
plt.ylabel('Precipitation_satellite (mm)')
# Adjusting the position of the legend
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.6), ncol=6)
plt.show()

Calcular la precipitación anual promedio en la cuenca


In [ ]:
data_annual_average.mean()

Calcular la precipitación mensual


In [ ]:
data_monthly = precipitation_satellite.resample('M',label='right',closed='right').sum()
data_monthly

Calcular la precipitación mensual


In [ ]:
data_monthly_mean_pixels =  data_monthly.mean(axis=1)
data_monthly_mean_pixels

Graficar

In [ ]:
fig, ax = plt.subplots(figsize=(40,5))
# Assuming dataset is a pandas DataFrame with labeled columns
data_monthly_mean_pixels.plot(kind='bar', ax=ax)
# Adding labels for the legend
ax.legend(title='Legend Title')
# Adding a label to the y-axis
plt.ylabel('Precipitation_satellite (mm)')
# Adjusting the position of the legend
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.6), ncol=6)
plt.show()

Calcular la precipitación mensual promedio (promedio de todos los píxeles en la cuenca)


In [ ]:
data_monthly_mean= data_monthly_mean_pixels.groupby(data_monthly_mean_pixels.index.month).mean()
data_monthly_mean

Graficar

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
# Assuming dataset is a pandas DataFrame with labeled columns
data_monthly_mean.plot(kind='bar', ax=ax)
# Adding labels for the legend
ax.legend(title='Legend Title')
# Adding a label to the y-axis
plt.ylabel('Precipitation_satellite (mm)')
plt.show()

### Importar precipitación in-situ 🌧️📍


Usemos tres pluviómetros instalados dentro de la cuenca

#### Para el pluviómetro 1 🌧️
Importar y preprocesar los datos.


In [ ]:
folder_pcp_1 = folder+'Rain_gauge_1/'
df_pcp_1= pd.read_table(folder_pcp_1+'Rain_gauge_1.csv', sep=',')
df_pcp_1.rename(columns={'Texas_tip_corrected_mm':'Pluviómetro_1'},inplace=True)
df_pcp_1.columns
df_pcp_1

Operaciones para organizar la información en un dataframe manejable


In [ ]:
# Rename the column 'Date_yy/mm/dd_hh:mm:ss' to 'Date'
df_pcp_1.rename(columns={'Date_yy/mm/dd_hh:mm:ss': 'Date'}, inplace=True)
# Convert the 'Date' column to datetime format
df_pcp_1['Date'] = df_pcp_1['Date'].apply(lambda x: pd.to_datetime(x, dayfirst=True))
# Set the 'Date' column as the index
df_pcp_1.set_index('Date', inplace=True)
df_pcp_1 = df_pcp_1[~df_pcp_1.index.duplicated(keep='first')]

df_pcp_1 = df_pcp_1.sort_index()

df_pcp_1

Graficar el año 2020 de la serie de precipitación importada


In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
# Assuming dataset is a pandas DataFrame with labeled columns
df_pcp_1.loc['2020'].plot(ax=ax)
# Adding labels for the legend
ax.legend(title='Legend Title')
# Adding a label to the y-axis
plt.ylabel('Precipitation_in situ (mm)')
# Adjusting the position of the legend
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.6), ncol=6)
plt.show()

Graficar la precipitación acumulada de 2020 a partir de la serie temporal importada


In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
# Assuming dataset is a pandas DataFrame with labeled columns
df_pcp_1.loc['2020'].cumsum().plot(ax=ax)
# Adding labels for the legend
ax.legend(title='Legend Title')
# Adding a label to the y-axis
plt.ylabel('Precipitation_satellite (mm)')
# Adjusting the position of the legend
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.6), ncol=6)
plt.show()

#### Para el pluviómetro 2 🌧️

Importar y preprocesar los datos.


In [ ]:
folder_pcp_2 = folder+'Rain_gauge_2/'
df_pcp_2= pd.read_table(folder_pcp_2+'Rain_gauge_2.csv', sep=',')
df_pcp_2

Operaciones para crear un dataframe manejable

In [ ]:
df_pcp_2['Date'] = df_pcp_2.Date.apply(lambda x: pd.to_datetime(x,dayfirst=True))
df_pcp_2.set_index(df_pcp_2['Date'],inplace=True)
df_pcp_2.rename(columns={'Precipitation':'Pluviómetro_2'},inplace=True)
df_pcp_2 = df_pcp_2.drop(labels='Date', axis=1)
df_pcp_2 = df_pcp_2[~df_pcp_2.index.duplicated(keep='first')]

df_pcp_2 = df_pcp_2.sort_index()
df_pcp_2

Graficar el año 2020 de la serie importada


In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
# Assuming dataset is a pandas DataFrame with labeled columns
df_pcp_2.loc['2020'].plot(ax=ax)
# Adding labels for the legend
ax.legend(title='Legend Title')
# Adding a label to the y-axis
plt.ylabel('Precipitation_in situ (mm)')
# Adjusting the position of the legend
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.6), ncol=6)
plt.show()

Graficar la precipitación acumulada del año 2020 a partir de la serie importada


In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
# Assuming dataset is a pandas DataFrame with labeled columns
df_pcp_2.loc['2020'].cumsum().plot(ax=ax)
# Adding labels for the legend
ax.legend(title='Legend Title')
# Adding a label to the y-axis
plt.ylabel('Precipitation_satellite (mm)')
# Adjusting the position of the legend
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.6), ncol=6)
plt.show()

#### Para el pluviómetro 3 🌧️

Importar y preprocesar los datos.

In [ ]:
folder_pcp_3 = folder+'Rain_gauge_3/'
df_pcp_3= pd.read_table(folder_pcp_3+'Rain_gauge_3.csv', sep=',')
df_pcp_3

Operaciones para crear un dataframe manejable


In [ ]:
df_pcp_3['Fecha'] = df_pcp_3.Fecha.apply(lambda x: pd.to_datetime(x,dayfirst=True))
df_pcp_3.set_index(df_pcp_3['Fecha'],inplace=True)
df_pcp_3 = df_pcp_3.drop(labels='Fecha', axis=1)
df_pcp_3.rename(columns={'Precipitation':'Pluviómetro_3'},inplace=True)
df_pcp_3 = df_pcp_3[~df_pcp_3.index.duplicated(keep='first')]

df_pcp_3 = df_pcp_3.sort_index()
df_pcp_3

Graficar la precipitación del año 2020


In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
# Assuming dataset is a pandas DataFrame with labeled columns
df_pcp_3.loc['2020'].plot(ax=ax)
# Adding labels for the legend
ax.legend(title='Legend Title')
# Adding a label to the y-axis
plt.ylabel('Precipitation_in situ (mm)')
# Adjusting the position of the legend
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.6), ncol=6)
plt.show()

Graficar la precipitación acumulada del año 2020


In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
# Assuming dataset is a pandas DataFrame with labeled columns
df_pcp_3.loc['2020'].cumsum().plot(ax=ax)
# Adding labels for the legend
ax.legend(title='Legend Title')
# Adding a label to the y-axis
plt.ylabel('Precipitation_satellite (mm)')
# Adjusting the position of the legend
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.6), ncol=6)
plt.show()

#### Comparar la precipitación in-situ 🌧️📍


Re-muestrear los datos de los 3 pluviómetros a escalas mensuales


In [ ]:
df_pcp_1_monthly = df_pcp_1.resample('M',label='right',closed='right').sum()
df_pcp_1_monthly= df_pcp_1_monthly.groupby(df_pcp_1_monthly.index.month).mean()
df_pcp_2_monthly = df_pcp_2.resample('M',label='right',closed='right').sum()
df_pcp_2_monthly= df_pcp_2_monthly.groupby(df_pcp_2_monthly.index.month).mean()
df_pcp_3_monthly = df_pcp_3.resample('M',label='right',closed='right').sum()
df_pcp_3_monthly= df_pcp_3_monthly.groupby(df_pcp_3_monthly.index.month).mean()
all_pcp_monthly = pd.concat([df_pcp_1_monthly, df_pcp_2_monthly, df_pcp_3_monthly], axis=1)
all_pcp_monthly

Graficar

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
# Assuming dataset is a pandas DataFrame with labeled columns
all_pcp_monthly.plot(kind='bar',ax=ax)
# Adding labels for the legend
ax.legend(title='Legend Title')
# Adding a label to the y-axis
plt.ylabel('Precipitation_in situ (mm)')
# Adjusting the position of the legend
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.6), ncol=6)
plt.show()

Ahora comparar con la precipitacón satelital

In [ ]:
monthly_satellite_mean = precipitation_satellite.resample('M').sum().mean(axis=1)
monthly_satellite_mean
#Agrupa por mes para obtener un valor promedio por cada mes del año
monthly_satellite_mean_by_month = monthly_satellite_mean.groupby(monthly_satellite_mean.index.month).mean()
monthly_satellite_mean_by_month


In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))

# Convertimos los índices a enteros explícitamente
x = list(range(1, 13))

# Graficar barras de pluviómetros
ax.bar(x, all_pcp_monthly.iloc[:, 0], width=0.2, label='Pluviómetro_1', align='center')
ax.bar([i + 0.2 for i in x], all_pcp_monthly.iloc[:, 1], width=0.2, label='Pluviómetro_2', align='center')
ax.bar([i + 0.4 for i in x], all_pcp_monthly.iloc[:, 2], width=0.2, label='Pluviómetro_3', align='center')

# Graficar la línea satelital alineada con el primer grupo de barras
ax.plot(x, monthly_satellite_mean_by_month.values,
        color='black', linewidth=2, linestyle='--', marker='o', label='Satélite (promedio)')

# Ejes y leyenda
ax.set_xticks([i + 0.2 for i in x])  # Centrar los labels bajo el grupo de barras
ax.set_xticklabels(x)
ax.set_ylabel('Precipitación (mm)')
ax.legend(title='Precipitación', loc='upper center', bbox_to_anchor=(0.5, -0.4), ncol=4)
plt.show()


## Importar datos de caudal a la salida de la cuenca de montaña 💧


Importar y organizar los datos de escorrentía en un dataframe manejable


In [ ]:
folder_runoff = folder+'Runoff_catchment_1/'
df_runoff =  pd.read_excel(folder_runoff+'Runoff_catchment_1.xlsx')
df_runoff['Fecha'] = df_runoff.Fecha.apply(lambda x: pd.to_datetime(x,dayfirst=True))
df_runoff.set_index(df_runoff['Fecha'],inplace=True)
df_runoff = df_runoff.drop(labels='Fecha', axis=1)
df_runoff

Graficar

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
# Assuming dataset is a pandas DataFrame with labeled columns
df_runoff.plot(ax=ax)
# Adding labels for the legend
ax.legend(title='Legend Title')
# Adding a label to the y-axis
plt.ylabel('Caudal ($m^3/s$)')
# Adjusting the position of the legend
plt.legend()
plt.show()


## Combinar datos de precipitación (pluviómetros + satélite) y escorrentía para la cuenca 🌧️💧⛰️


In [ ]:
df_pcp_1_daily = df_pcp_1.resample('D',label='right',closed='right').sum()
df_pcp_2_daily = df_pcp_2.resample('D',label='right',closed='right').sum()
df_pcp_3_daily = df_pcp_3.resample('D',label='right',closed='right').sum()
df_runoff_daily = df_runoff.resample('D',label='right',closed='right').mean()
all_data_daily = pd.concat([df_pcp_1_daily, df_pcp_2_daily, df_pcp_3_daily, precipitation_satellite, df_runoff_daily], axis=1)
all_data_daily

### Determinar periodos con datos concurrentes

In [ ]:
concurrent_periods = all_data_daily.dropna().index

# Create a figure and axis
fig, ax = plt.subplots(figsize=(15, 6))

# Loop through columns
for i, col in enumerate(all_data_daily.columns):
    # Get a boolean mask where data is not NaN for the current column
    mask = ~all_data_daily[col].isna()

    # Get the indices of True values in the mask
    indices = np.where(mask)[0]

    # Plot horizontal lines for continuity
    ax.hlines(i, indices[0], indices[-1], colors='0.1', linewidth=5, label=col)

# Set y-ticks and labels
ax.set_yticks(range(len(all_data_daily.columns)))
ax.set_yticklabels(all_data_daily.columns)

# Set x-axis label
ax.set_xlabel('Date')

# Set the x-axis ticks to show years
years = pd.to_datetime(all_data_daily.index).year
unique_years = np.unique(years)
ax.set_xticks(np.arange(len(all_data_daily.index), step=365))
ax.set_xticklabels(unique_years,rotation=45)

# Add legend
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=11)

# Show the plot
plt.show()


## Dividir los datos en periodos de entrenamiento y prueba


In [ ]:
all_data_daily = all_data_daily[~(all_data_daily.isna().any(axis=1) | (all_data_daily.lt(0).any(axis=1)))]
input_data_train = np.array(all_data_daily['2013':'2019'].iloc[:,:-1])
input_data_test = np.array(all_data_daily['2020':'2021-06'].iloc[:,:-1])

In [ ]:
input_data_train

In [ ]:
output_data_train = np.reshape(np.array(all_data_daily['2013':'2019'].iloc[:,-1]),(all_data_daily['2013':'2019'].shape[0],1))
output_data_test = np.reshape(np.array(all_data_daily['2020':'2021-06'].iloc[:,-1]),(all_data_daily['2020':'2021-06'].shape[0],1))

In [ ]:
output_data_train

In [ ]:
input_data_train_lags, output_data_train_lags= lagged_dataset(input_data_train, 3, output_data_train,15)

In [ ]:
input_data_train_lags

In [ ]:
output_data_train_lags

In [ ]:
input_data_test_lags, output_data_test_lags= lagged_dataset(input_data_test, 3, output_data_test,15)

In [ ]:
input_data_test_lags

In [ ]:
output_data_test_lags

## Creación y entrenamiento de un modelo Random Forest (sin pronóstico)

### Definir los hiperparámetros del modelo ⚙️


In [ ]:
min_samples_splt=10
min_samples_lf=4
max_dpth=350
n_trees=600
max_ft='sqrt'

### Definir el modelo 🧠


In [ ]:
regr=RandomForestRegressor(bootstrap=True,min_samples_split=min_samples_splt,
                               max_depth=max_dpth,max_features=max_ft,
                               min_samples_leaf=min_samples_lf,
                               n_estimators=n_trees,oob_score=True,n_jobs=-1,
                               warm_start=True,random_state=22)

### Entrenar el modelo 🎯


In [ ]:
# Correcting the shape of output_data_train_lags
regr = regr.fit(input_data_train_lags, output_data_train_lags.ravel())

### Generar simulaciones para el periodo de entrenamiento 🧪


In [ ]:
simulations_data_train= regr.predict(input_data_train_lags)
simulations_data_train= np.reshape(simulations_data_train, (-1, 1))
simulations_data_train

### Generar simulaciones para el periodo de prueba


In [ ]:
#Prediction on unseen data
simulations_data_test= regr.predict(input_data_test_lags)
simulations_data_test= np.reshape(simulations_data_test, (-1, 1))
simulations_data_test

### Evaluación del modelo

Calcular los coeficientes de correlación para los periodos de entrenamiento y prueba.


In [ ]:
r2_test=regr.score(input_data_test_lags, output_data_test_lags)
r2_train=regr.score(input_data_train_lags, output_data_train_lags)
print(r2_train,r2_test)

## Creación y entrenamiento de un modelo Random Forest (con pronóstico)


### Caso de pronóstico a un día 📆➡️1️⃣


In [ ]:
leadtime = 1
input_data_train_lags, output_data_train_lags= lagged_dataset_pron(input_data_train, 7, output_data_train,15, lead_time=leadtime)
input_data_test_lags, output_data_test_lags= lagged_dataset_pron(input_data_test, 7, output_data_test,15, lead_time=leadtime)
min_samples_splt=10
min_samples_lf=4
max_dpth=350
n_trees=600
max_ft='sqrt'
regr=RandomForestRegressor(bootstrap=True,min_samples_split=min_samples_splt,
                               max_depth=max_dpth,max_features=max_ft,
                               min_samples_leaf=min_samples_lf,
                               n_estimators=n_trees,oob_score=True,n_jobs=-1,
                               warm_start=True,random_state=42)
regr=regr.fit(input_data_train_lags, output_data_train_lags.ravel())
#Prediction on training data
simulations_data_train= regr.predict(input_data_train_lags)
simulations_data_train= np.reshape(simulations_data_train, (-1, 1))
#Prediction on unseen data
simulations_data_test= regr.predict(input_data_test_lags)
simulations_data_test= np.reshape(simulations_data_test, (-1, 1))
r2_test=regr.score(input_data_test_lags, output_data_test_lags)
r2_train=regr.score(input_data_train_lags, output_data_train_lags)
print(r2_train,r2_test)

### Pronósticos en el periodo de prueba 📊⏳


In [ ]:
simulations_data_test

### Evaluación usando una combinación de métricas de eficiencia

In [ ]:
kge, rmse, pbias , r2 = calculate_hydro_metrics(simulations_data_test, output_data_test_lags)
print(f"RMSE: {rmse[0]:.4f}")
print(f"PBias: {pbias[0]:.4f}")
print(f"KGE: {kge[0]:.4f}")
print(f"R2: {r2:.4f}")

### Evaluación mediante inspección visual

### Pronósticos de escorrentía a un día


In [ ]:
simulations_data_test = pd.DataFrame(simulations_data_test, columns=['Forecasts'], index=all_data_daily['2019':'2021-06'].index[-len(simulations_data_test):])
simulations_data_test

### Y las observaciones de escorrentía


In [ ]:
observations_data_test = pd.DataFrame(output_data_test_lags, columns=['Observations'], index=all_data_daily['2019':'2021-06'].index[-len(output_data_test_lags):])
observations_data_test

### Combinar pronósticos y observaciones en un DataFrame


In [ ]:
testing_period = pd.concat([simulations_data_test, observations_data_test], axis=1)

In [ ]:
testing_period

### Graficar (comparar) pronósticos y observaciones


In [ ]:

# Calculate mean and percentiles
mean_obs = testing_period['Observations'].mean()
p05 = testing_period['Observations'].quantile(0.05)
p95= testing_period['Observations'].quantile(0.95)

# Create the plot
fig, ax = plt.subplots(figsize=(10, 5))

# Plot forecasts and observations
testing_period['Forecasts'].plot(ax=ax, color='red', marker='o', linestyle='', markersize=2, label='Forecasts')
testing_period['Observations'].plot(ax=ax, color='black', linestyle='-', label='Observations')

# Add horizontal lines for mean and percentiles
ax.axhline(mean_obs, color='blue', linestyle='--', linewidth=1, label=f'Mean ({mean_obs:.2f} $m^3/s$)')
ax.axhline(p05, color='green', linestyle=':', linewidth=1, label=f'5th Percentile ({p05:.2f} $m^3/s$)')
ax.axhline(p95, color='orange', linestyle=':', linewidth=1, label=f'95th Percentile ({p95:.2f} $m^3/s$)')

# Add labels and legend
ax.set_ylabel('Runoff ($m^3/s$)')
ax.legend(loc='upper right')

# Display the plot
plt.show()


### Diagrama de dispersión de pronósticos y observaciones


In [ ]:
from scipy.stats import gaussian_kde


# Step 1: Clean the data to handle NaN and Inf values
testing_period = testing_period.replace([np.inf, -np.inf], np.nan).dropna(subset=['Observations', 'Forecasts'])

# Step 2: Scatter plot data
x = testing_period['Observations'].values
y = testing_period['Forecasts'].values

# Step 3: Create the figure and axis
fig, ax = plt.subplots(figsize=(6, 6))

# Scatter plot for Observations vs Forecasts
sns.scatterplot(x=x, y=y, color='red', marker='o', s=30, ax=ax)

# Step 4: KDE using scipy's gaussian_kde
# Create grid for KDE
xmin, xmax = x.min(), x.max()
ymin, ymax = y.min(), y.max()
xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
positions = np.vstack([xx.ravel(), yy.ravel()])

# Perform KDE
kde = gaussian_kde(np.vstack([x, y]))
density = np.reshape(kde(positions).T, xx.shape)

# Plot KDE contours
ax.contour(xx, yy, density, levels=13, cmap='magma')

# Step 5: Add bisector line (y = x)
min_val = min(xmin, ymin)
max_val = max(xmax, ymax)
ax.plot([min_val, max_val], [min_val, max_val], color='blue', linestyle='--', label='Bisector Line')

# Step 6: Add labels and legend
ax.set_xlabel('Observaciones ($m^3/s$)')
ax.set_ylabel('Pronósticos ($m^3/s$)')

# Show the plot
plt.show()




## Incluir datos de ENSO

https://psl.noaa.gov/gcos_wgsp/Timeseries/


Importar datos


In [ ]:
# Define the path to
folder_nino12 = folder+'ENSO/nino12.long.anom.data.xlsx'
folder_nino3 = folder+'ENSO/nino3.long.anom.data.xlsx'
folder_nino34 = folder+'ENSO/nino34.long.anom.data.xlsx'


In [ ]:
nino12 =  pd.read_excel(folder_nino12)
nino3 =  pd.read_excel(folder_nino3)
nino34 =  pd.read_excel(folder_nino34)

In [ ]:
# Melt the DataFrame to convert it to long format
nino12_long = nino12.melt(id_vars=['Year'], var_name='Month', value_name='Data')
# Replace '-99.99' values with NaN
nino12_long['Data'] = nino12_long['Data'].replace(-99.99, np.nan)
# Convert 'Year' and 'Month' to datetime format
nino12_long['Date'] = pd.to_datetime(nino12_long['Year'].astype(str) + '-' + nino12_long['Month'], format='%Y-%B')
# Set 'Date' as the index
nino12_time_series = nino12_long.set_index('Date')[['Data']]
# Display the resulting DataFrame
nino12_time_series

# Melt the DataFrame to convert it to long format
nino3_long = nino3.melt(id_vars=['Year'], var_name='Month', value_name='Data')
# Replace '-99.99' values with NaN
nino3_long['Data'] = nino3_long['Data'].replace(-99.99, np.nan)
# Convert 'Year' and 'Month' to datetime format
nino3_long['Date'] = pd.to_datetime(nino3_long['Year'].astype(str) + '-' + nino3_long['Month'], format='%Y-%B')
# Set 'Date' as the index
nino3_time_series = nino3_long.set_index('Date')[['Data']]
# Display the resulting DataFrame
nino3_time_series

# Melt the DataFrame to convert it to long format
nino34_long = nino34.melt(id_vars=['Year'], var_name='Month', value_name='Data')
# Replace '-99.99' values with NaN
nino34_long['Data'] = nino34_long['Data'].replace(-99.99, np.nan)
# Convert 'Year' and 'Month' to datetime format
nino34_long['Date'] = pd.to_datetime(nino34_long['Year'].astype(str) + '-' + nino34_long['Month'], format='%Y-%B')
# Set 'Date' as the index
nino34_time_series = nino34_long.set_index('Date')[['Data']]
# Display the resulting DataFrame
nino34_time_series

### Convertir datos mensuales a datos diarios

In [ ]:
nino12_df = nino12_time_series.resample('D').ffill()
nino3_df = nino3_time_series.resample('D').ffill()
nino34_df = nino34_time_series.resample('D').ffill()

In [ ]:
ENSO_daily = pd.concat([nino12_df,nino3_df,nino34_df], axis=1)
ENSO_daily

### Combine all Information

In [ ]:
all_data_daily_ENSO = pd.concat([all_data_daily, ENSO_daily], axis=1)
all_data_daily_ENSO

In [ ]:
all_data_daily_ENSO.loc['2013']

### Define training and testing periods

In [ ]:
all_data_daily_ENSO = all_data_daily_ENSO[~(all_data_daily_ENSO.isna().any(axis=1))]
all_data_daily_ENSO.shape
inputs = all_data_daily_ENSO.drop(all_data_daily_ENSO.columns[-4], axis=1)
input_data_train = np.array(inputs['2013':'2019'].iloc[:,:-1])
input_data_test = np.array(inputs['2020':'2021-06'].iloc[:,:-1])
output_data_train = np.reshape(np.array(all_data_daily_ENSO['2013':'2019'].iloc[:,-4]),(all_data_daily_ENSO['2013':'2019'].shape[0],1))
output_data_test = np.reshape(np.array(all_data_daily_ENSO['2020':'2021-06'].iloc[:,-4]),(all_data_daily_ENSO['2020':'2021-06'].shape[0],1))

In [ ]:
input_data_train

In [ ]:
output_data_test

## Desarrollo de modelos de pronóstico a un día

In [ ]:
leadtime = 1
input_data_train_lags, output_data_train_lags= lagged_dataset_pron(input_data_train, 3, output_data_train,15, lead_time=leadtime)
input_data_test_lags, output_data_test_lags= lagged_dataset_pron(input_data_test, 3, output_data_test,15, lead_time=leadtime)
min_samples_splt=10
min_samples_lf=4
max_dpth=350
n_trees=600
max_ft='sqrt'
regr=RandomForestRegressor(bootstrap=True,min_samples_split=min_samples_splt,
                               max_depth=max_dpth,max_features=max_ft,
                               min_samples_leaf=min_samples_lf,
                               n_estimators=n_trees,oob_score=True,n_jobs=-1,
                               warm_start=True,random_state=22)
regr=regr.fit(input_data_train_lags, output_data_train_lags.ravel())
#Prediction on training data
simulations_data_train_ENSO= regr.predict(input_data_train_lags)
simulations_data_train_ENSO= np.reshape(simulations_data_train_ENSO, (-1, 1))
#Prediction on unseen data
simulations_data_test_ENSO= regr.predict(input_data_test_lags)
simulations_data_test_ENSO= np.reshape(simulations_data_test_ENSO, (-1, 1))
r2_test=regr.score(input_data_test_lags, output_data_test_lags)
r2_train=regr.score(input_data_train_lags, output_data_train_lags)
print(r2_train,r2_test)

### Evaluation with efficiency metrics

In [ ]:
kge, rmse, pbias , r2 = calculate_hydro_metrics(simulations_data_test_ENSO, output_data_test_lags)
print(f"RMSE: {rmse[0]:.4f}")
print(f"PBias: {pbias[0]:.4f}")
print(f"KGE: {kge[0]:.4f}")
print(f"R2: {r2:.4f}")

### Inspección visual

In [ ]:
simulations_data_test_ENSO = pd.DataFrame(simulations_data_test, columns=['Forecasts'], index=all_data_daily['2019':'2021-06'].index[-len(simulations_data_test):])
observations_data_test_ENSO = pd.DataFrame(output_data_test_lags, columns=['Observations'], index=all_data_daily['2019':'2021-06'].index[-len(output_data_test_lags):])
testing_period_ENSO = pd.concat([simulations_data_test_ENSO, observations_data_test_ENSO], axis=1)


# Calculate mean and percentiles for Observations
mean_obs = testing_period_ENSO['Observations'].mean()
p05 = testing_period_ENSO['Observations'].quantile(0.05)
p95 = testing_period_ENSO['Observations'].quantile(0.95)

# Create the plot
fig, ax = plt.subplots(figsize=(10,5))

# Plot Forecasts and Observations
testing_period_ENSO['Forecasts'].plot(ax=ax, color='red', marker='o', linestyle='', markersize=2, label='Forecasts')
testing_period_ENSO['Observations'].plot(ax=ax, color='black', linestyle='-', label='Observations')

# Add horizontal lines for mean and percentiles
ax.axhline(mean_obs, color='blue', linestyle='--', linewidth=1, label=f'Mean ({mean_obs:.2f} $m^3/s$)')
ax.axhline(p05, color='green', linestyle=':', linewidth=1, label=f'5th Percentile ({p05:.2f} $m^3/s$)')
ax.axhline(p95, color='orange', linestyle=':', linewidth=1, label=f'95th Percentile ({p95:.2f} $m^3/s$)')

# Adding labels for the legend
ax.legend(title='Legend Title', loc='upper right')

# Adding a label to the y-axis
ax.set_ylabel('Caudal ($m^3/s$)')

# Display the plot
plt.show()

In [ ]:

# Step 1: Clean the data (replace inf with NaN, drop NaNs)
testing_period_ENSO = testing_period_ENSO.replace([np.inf, -np.inf], np.nan).dropna(subset=['Observations', 'Forecasts'])

# Step 2: Scatter plot data
x = testing_period_ENSO['Observations'].values
y = testing_period_ENSO['Forecasts'].values

# Create the figure and axis
fig, ax = plt.subplots(figsize=(6, 6))

# Scatter plot for Observations vs Forecasts
sns.scatterplot(x=x, y=y, color='red', marker='o', s=30, ax=ax)

# Step 3: KDE using scipy's gaussian_kde
# Create grid for KDE
xmin, xmax = x.min(), x.max()
ymin, ymax = y.min(), y.max()
xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
positions = np.vstack([xx.ravel(), yy.ravel()])

# Perform KDE
kde = gaussian_kde(np.vstack([x, y]))
density = np.reshape(kde(positions).T, xx.shape)

# Plot KDE contours
ax.contour(xx, yy, density, levels=13, cmap='magma')

# Step 4: Add bisector line (y = x)
min_val = min(xmin, ymin)
max_val = max(xmax, ymax)
ax.plot([min_val, max_val], [min_val, max_val], color='blue', linestyle='--', label='Bisector Line')

# Step 5: Add labels and legend
ax.set_xlabel('Observaciones ($m^3/s$)')
ax.set_ylabel('Pronósticos ($m^3/s$)')

# Show the plot
plt.show()

## Ajuste de hiperparámetros del modelo de pronóstico ⚙️🔍


### Definir el dominio de búsqueda de hiperparámetros 🎯📌


In [ ]:
# Define the parameter grid
param_grid = {
    'min_samples_split': [ 10, 20],
    'min_samples_leaf': [2, 10],
    'max_depth': [100, 300],
    'n_estimators': [100, 300],
    'max_features': ['sqrt','log2']
}

# Calculate the total number of combinations
total_combinations = len(list(itertools.product(*param_grid.values())))

total_combinations

### Buscar la mejor combinación de hiperparámetros


In [ ]:
# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=RandomForestRegressor(oob_score=True, n_jobs=-1, warm_start=True),
                           param_grid=param_grid, cv=3, n_jobs=-1, scoring='r2')

# Fit the GridSearchCV to your data
grid_search.fit(input_data_train_lags, output_data_train_lags.ravel())

# Get the best hyperparameters
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters:")
print(best_params)


### Una hiperparametrización más rigurosa 🧪⚙️









In [ ]:
# Define the parameter grid
param_grid = {
    'min_samples_split': [5, 10, 20],
    'min_samples_leaf': [2, 4, 8],
    'max_depth': [100, 200, 350],
    'n_estimators': [200, 300, 400, 500, 600],
    'max_features': ['auto', 'sqrt','log2']
}
# Calculate the total number of combinations
total_combinations = len(list(itertools.product(*param_grid.values())))

total_combinations
# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=RandomForestRegressor(oob_score=True, n_jobs=-1, warm_start=True),
                           param_grid=param_grid, cv=3, n_jobs=-1, scoring='r2')

# Fit the GridSearchCV to your data
grid_search.fit(input_data_train_lags, output_data_train_lags.ravel())

# Get the best hyperparameters
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters:")
print(best_params)

In [ ]:
best_model
simulations_data_train_ENSO= best_model.predict(input_data_train_lags)
simulations_data_train_ENSO= np.reshape(simulations_data_train_ENSO, (-1, 1))
#Prediction on unseen data
simulations_data_test_ENSO= best_model.predict(input_data_test_lags)
simulations_data_test_ENSO= np.reshape(simulations_data_test_ENSO, (-1, 1))
#Nash_Sutcliffe
r2_test=regr.score(input_data_test_lags, output_data_test_lags)
r2_train=regr.score(input_data_train_lags, output_data_train_lags)
print(r2_train,r2_test)

In [ ]:
best_model


In [ ]:
leadtime = 1
input_data_train_lags, output_data_train_lags= lagged_dataset_pron(input_data_train, 3, output_data_train,15, lead_time=leadtime)
input_data_test_lags, output_data_test_lags= lagged_dataset_pron(input_data_test, 3, output_data_test,15, lead_time=leadtime)
min_samples_splt=10
min_samples_lf=2
max_dpth=300
n_trees=300
max_ft='sqrt'
regr=RandomForestRegressor(bootstrap=True,min_samples_split=min_samples_splt,
                               max_depth=max_dpth,max_features=max_ft,
                               min_samples_leaf=min_samples_lf,
                               n_estimators=n_trees,oob_score=True,n_jobs=-1,
                               warm_start=True,random_state=22)
regr=regr.fit(input_data_train_lags, output_data_train_lags.ravel())
#Prediction on training data
simulations_data_train_ENSO= regr.predict(input_data_train_lags)
simulations_data_train_ENSO= np.reshape(simulations_data_train_ENSO, (-1, 1))
#Prediction on unseen data
simulations_data_test_ENSO= regr.predict(input_data_test_lags)
simulations_data_test_ENSO= np.reshape(simulations_data_test_ENSO, (-1, 1))
r2_test=regr.score(input_data_test_lags, output_data_test_lags)
r2_train=regr.score(input_data_train_lags, output_data_train_lags)
print(r2_train,r2_test)
kge, rmse, pbias , r2 = calculate_hydro_metrics(simulations_data_test_ENSO, output_data_test_lags)
print(f"RMSE: {rmse[0]:.4f}")
print(f"PBias: {pbias[0]:.4f}")
print(f"KGE: {kge[0]:.4f}")
print(f"R2: {r2:.4f}")

In [ ]:
simulations_data_test_ENSO = pd.DataFrame(simulations_data_test, columns=['Forecasts'], index=all_data_daily['2019':'2021-06'].index[-len(simulations_data_test):])
observations_data_test_ENSO = pd.DataFrame(output_data_test_lags, columns=['Observations'], index=all_data_daily['2019':'2021-06'].index[-len(output_data_test_lags):])
testing_period_ENSO = pd.concat([simulations_data_test_ENSO, observations_data_test_ENSO], axis=1)


# Calculate mean and percentiles for Observations
mean_obs = testing_period_ENSO['Observations'].mean()
p05 = testing_period_ENSO['Observations'].quantile(0.05)
p95 = testing_period_ENSO['Observations'].quantile(0.95)

# Create the plot
fig, ax = plt.subplots(figsize=(10,5))

# Plot Forecasts and Observations
testing_period_ENSO['Forecasts'].plot(ax=ax, color='red', marker='o', linestyle='', markersize=2, label='Forecasts')
testing_period_ENSO['Observations'].plot(ax=ax, color='black', linestyle='-', label='Observations')

# Add horizontal lines for mean and percentiles
ax.axhline(mean_obs, color='blue', linestyle='--', linewidth=1, label=f'Mean ({mean_obs:.2f} $m^3/s$)')
ax.axhline(p05, color='green', linestyle=':', linewidth=1, label=f'5th Percentile ({p05:.2f} $m^3/s$)')
ax.axhline(p95, color='orange', linestyle=':', linewidth=1, label=f'95th Percentile ({p95:.2f} $m^3/s$)')

# Adding labels for the legend
ax.legend(title='Legend Title', loc='upper right')

# Adding a label to the y-axis
ax.set_ylabel('Caudal ($m^3/s$)')

# Display the

In [ ]:
# Step 1: Clean the data (replace inf with NaN, drop NaNs)
testing_period_ENSO = testing_period_ENSO.replace([np.inf, -np.inf], np.nan).dropna(subset=['Observations', 'Forecasts'])

# Step 2: Scatter plot data
x = testing_period_ENSO['Observations'].values
y = testing_period_ENSO['Forecasts'].values

# Create the figure and axis
fig, ax = plt.subplots(figsize=(6, 6))

# Scatter plot for Observations vs Forecasts
sns.scatterplot(x=x, y=y, color='red', marker='o', s=30, ax=ax)

# Step 3: KDE using scipy's gaussian_kde
# Create grid for KDE
xmin, xmax = x.min(), x.max()
ymin, ymax = y.min(), y.max()
xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
positions = np.vstack([xx.ravel(), yy.ravel()])

# Perform KDE
kde = gaussian_kde(np.vstack([x, y]))
density = np.reshape(kde(positions).T, xx.shape)

# Plot KDE contours
ax.contour(xx, yy, density, levels=13, cmap='magma')

# Step 4: Add bisector line (y = x)
min_val = min(xmin, ymin)
max_val = max(xmax, ymax)
ax.plot([min_val, max_val], [min_val, max_val], color='blue', linestyle='--', label='Bisector Line')

# Step 5: Add labels and legend
ax.set_xlabel('Observaciones ($m^3/s$)')
ax.set_ylabel('Pronósticos ($m^3/s$)')

# Show the plot
plt.show()